<a href="https://colab.research.google.com/github/tanmalaker/GEE/blob/main/DownladNAIPImage_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install geemap
import ee
import geemap

# Authenticate and initialize
ee.Authenticate()
ee.Initialize(project='ee-malakerku')

In [32]:
##Uploading shapefile from content_ As ZIP
from google.colab import files
uploaded = files.upload()

Saving Brazoria.zip to Brazoria (1).zip


In [33]:
import ee
import geemap
import zipfile
import geopandas as gpd


# === Step 1: Unzip shapefile (assumes shapefile is zipped) ===
zip_path = "/content/Brazoria.zip"
extract_to = "/content/Brazoria"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# === Step 2: Read shapefile using geopandas ===
gdf = gpd.read_file(extract_to)

# === Step 3: Convert first geometry to ee.Geometry ===
roi_geometry = gdf.geometry[0]  # First feature
roi_json = roi_geometry.__geo_interface__  # GeoJSON format
roi = ee.Geometry(roi_json)

# === Step 4: Create interactive map ===
Map = geemap.Map()
Map.centerObject(roi, 15)  # Zoom to ROI

# === Step 5: Add ROI to map ===
Map.addLayer(roi, {'color': 'red'}, 'ROI Boundary')
Map


EEException: The service is currently unavailable.

In [ ]:
import zipfile #Imporing the roi as zip to avoid missing file attributes

with zipfile.ZipFile("/content/BrazoriaBldFootprntROI.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Area")
import geopandas as gpd

gdf = gpd.read_file("/content/Area")  # Auto-detects .shp inside
roi_geometry = gdf.geometry[0]  # Select the first feature

# Convert to GeoJSON format
roi_json = roi_geometry.__geo_interface__  # Convert to GeoJSON format
roi = ee.Geometry(roi_json)


In [ ]:
#Roi incase have only shapefile
#  import geopandas as gpd
# roi_gdf = gpd.read_file('/path/to/your/shapefile.shp')  # upload to Colab first
# roi = geemap.geopandas_to_ee(roi_gdf)


In [ ]:

# Load and mosaic NAIP images (latest 2022–2024)
naip = ee.ImageCollection('USDA/NAIP/DOQQ') \
          .filterBounds(roi) \
          .filterDate('2020-01-01', '2025-12-31') \
          .sort('system:time_start', False) \
          .mosaic() \
          .clip(roi)


# Create map and center it on ROI
Map = geemap.Map()
Map.centerObject(roi, 14)

# Add layers
Map.addLayer(naip, {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255}, 'NAIP 2020–2025')
Map.addLayer(roi, {'color': 'red'}, 'ROI')
Map

In [ ]:
# Set export task includes NIR= 4bands, for the building footprint extraction we need RGB not 4 bands togather
task = ee.batch.Export.image.toDrive(
    image=naip,
    description='NAIP_Mosaic_Export',
    folder='GEE_Exports',
    fileNamePrefix='NAIP_Brazoria',
    region=roi,
    scale=0.6,  # native resolution
    maxPixels=1e13
)

# Start the export task
task.start()

# Check status (optional)
import time
print("Export started... Checking status every 30 seconds.")
while task.active():
    print(task.status())
    time.sleep(30)

print("Done.")
##Result will be a 4 band image, but show a greyscale in bareland and higher refelectance values. Solution is download RGB excluding NIR (3 band image)

Export started... Checking status every 30 seconds.
{'state': 'READY', 'description': 'NAIP_Mosaic_Export', 'priority': 100, 'creation_timestamp_ms': 1751468026840, 'update_timestamp_ms': 1751468026840, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'FJCACZH7WO3XCGTCL6VDM4UJ', 'name': 'projects/ee-malakerku/operations/FJCACZH7WO3XCGTCL6VDM4UJ'}
Done.


In [ ]:
# Select RGB only
naip_rgb = naip.select(['R', 'G', 'B'])

# Optional: Visualize for better-looking output
vis = naip_rgb.visualize(min=0, max=255)

# Export visualized image to Drive
task = ee.batch.Export.image.toDrive(
    image=vis,
    description='NAIP_Export_RGB',
    folder='LSBP',
    fileNamePrefix='BrazoriaBld_rgb',
    region=roi.bounds(),
    scale=0.6,
    maxPixels=1e13
)
task.start()
# Check status (optional)
import time
print("Export started... Checking status every 30 seconds.")
while task.active():
    print(task.status())
    time.sleep(30)

print("Done.")

Export started... Checking status every 30 seconds.
{'state': 'READY', 'description': 'NAIP_Export_RGB', 'priority': 100, 'creation_timestamp_ms': 1751468195263, 'update_timestamp_ms': 1751468195263, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '33ZADEJJX2ED6HMNAJWG5JMH', 'name': 'projects/ee-malakerku/operations/33ZADEJJX2ED6HMNAJWG5JMH'}
{'state': 'RUNNING', 'description': 'NAIP_Export_RGB', 'priority': 100, 'creation_timestamp_ms': 1751468195263, 'update_timestamp_ms': 1751468201480, 'start_timestamp_ms': 1751468201439, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': '33ZADEJJX2ED6HMNAJWG5JMH', 'name': 'projects/ee-malakerku/operations/33ZADEJJX2ED6HMNAJWG5JMH'}
Done.
